In [141]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
from urllib.parse import quote
import pandas as pd
from tqdm import tqdm
import pyperclip
import time

In [142]:
# 드라이버 설정
options = Options()
options.add_argument("--start-maximized")  # 창 최대화
driver = webdriver.Chrome(options=options)

url = "https://hotels.naver.com/"
driver.get(url)

In [143]:
# 찾고자하는 여행지 키워드 적어주세요.
keyword = "7호선 상동역"

In [144]:
# 어디로 가시나요? 클릭
locationSearch = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[1]/div[2]/div/div/div/div[1]/button'))
)

locationSearch.click()

# 키워드 카피 후 검색하기
pyperclip.copy(keyword)
locationName = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div/input'))
)

locationName.send_keys(Keys.CONTROL + 'v')

# 키워드 첫번째 요소 클릭 -> 조회엔터
# XPATH
# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[2]/section/ul/li[1]/a'))).click()
# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[1]/div[2]/div/div/button'))).click()

# CSS_SELECTOR

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a.SearchResults_anchor__eU0Fy'))).click()
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button.SearchBox_search__0Suoj'))).click()

In [145]:
review_data = []

def get_hotel_data():
    while True:
        # 호텔 목록이 로드될 때까지 대기
        hotelsList = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.SearchList_HotelItem__RRmaZ > a"))
        )
        
        # 호텔 목록의 개수가 14개가 아닌 경우, 마지막 페이지로 간주
        if len(hotelsList) < 14:
            print("마지막 페이지 발견")
            print(f"수집할 호텔 개수: {len(hotelsList)}")
        
        # 호텔 링크 수집
        hotel_links = [hotel.get_attribute('href') for hotel in hotelsList]
        
        # 각 호텔 페이지에서 데이터 수집
        for link in tqdm(hotel_links):  
            driver.execute_script("window.open(arguments[0]);", link)
            driver.switch_to.window(driver.window_handles[-1])  # 새 탭으로 전환
        
            try:
                time.sleep(2)  # 로딩 대기
                # BeautifulSoup 사용을 위해 페이지 소스 가져오기
                soup = bs(driver.page_source, 'html.parser')
                
                hotel_title = soup.select_one('h3 span').text.strip() if soup.select_one('h3 span') else 'Unknown'
                hotel_grade = soup.select_one('h3 div i').text.strip() if soup.select_one('h3 div i') else 'Unknown'
                
                # 네 번째 <li>의 <div class="wrap"> 아래 모든 <span> 태그 찾기
                rating_list = soup.select('ul.common_infoList__GR183 > li:nth-of-type(4) div.wrap span.common_txt__5usYI')
                
                # 평점 초기화
                hc_score = '0'
                bk_score = '0'
                ta_score = '0'
                
                # 모든 <li> 태그를 검색하여 평점 정보를 추출
                rating_items = soup.select('ul.common_infoList__GR183 > li')
                
                for item in rating_items:
                    # <i> 태그의 data-label 속성을 검사하여 평점 관련 항목인지 확인
                    data_label = item.find('i')['data-label']
                    if data_label == 'rating':
                        # 평점 정보를 담고 있는 <div class="wrap"> 아래의 <span> 태그 찾기
                        spans = item.select('div.wrap > span.common_txt__5usYI')
                        for span in spans:
                            text = span.get_text(strip=True)
                            if '호텔스컴바인' in text:
                                hc_score = span.find('b').get_text(strip=True) if span.find('b') else '0'
                            elif '부킹닷컴' in text:
                                bk_score = span.find('b').get_text(strip=True) if span.find('b') else '0'
                            elif '트립어드바이저' in text:
                                ta_score = span.find('b').get_text(strip=True) if span.find('b') else '0'
                # 실행
                sort_by_lowest_rating()
                
                # 리뷰 수집
                reviews = soup.select('ul.common_ReviewList__XlcWm > li.common_item__dEnNf')
                for review in reviews:
                    title = review.select_one('div.common_title_txt__0YGK_').text.strip() if review.select_one('div.common_title_txt__0YGK_') else ''
                    positive_feedback = review.select_one('p.common_as_positive__MaGjy').text.strip() if review.select_one('p.common_as_positive__MaGjy') else ''
                    negative_feedback = review.select_one('p.common_as_negative__cKs_1').text.strip() if review.select_one('p.common_as_negative__cKs_1') else ''
                    
                    # 리뷰를 하나의 문자열로 포맷팅
                    review_text = f"{title} {positive_feedback} {negative_feedback}"
                    
                review_data.append({
                    '호텔이름': hotel_title,
                    '호텔성급': hotel_grade,
                    '부킹닷컴평점': bk_score,
                    '호텔스컴바인평점': hc_score,
                    '트립어드바이저평점': ta_score,
                    '리뷰': review_text,
                    '링크': link
                })
                
            except Exception as e:
                print(f"오류 발생: {e}")
            finally:
                # 현재 탭 닫기
                driver.close()
                # 원래 탭으로 전환
                driver.switch_to.window(driver.window_handles[0])

        # 페이지네이션 다음 버튼 확인
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'button.Pagination_next__tTpJB'))
            )
            # 버튼이 비활성화되었는지 확인
            if 'disabled' in next_button.get_attribute('class') or not next_button.is_enabled():
                print("페이지네이션의 다음 버튼이 비활성화되었습니다. 데이터 수집을 종료합니다.")
                break
            else:
                next_button.click()
                time.sleep(3)  # 다음 페이지 로딩 대기
        except Exception as e:
            print(f"페이지네이션 오류 발생: {e}")
            break

In [146]:
# 평점 낮은 순으로 변경하는 함수
def sort_by_lowest_rating():
    try:
        # driver.switch_to.window(driver.window_handles[-1])  # 새 탭으로 전환
        
        # 이용 후기 탭 클릭
        review_tab = None
        tabs = driver.find_elements(By.CSS_SELECTOR, 'div.common_Tab__ZuTPd > div > a.common_item__ZZa4H')
        for tab in tabs:
            if '이용후기' in tab.text:
                review_tab = tab
                break
        
        if review_tab:
            review_tab.click()
            # 이용 후기 탭 클릭 후 드롭다운 메뉴가 나타날 때까지 대기
            try:
                WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.common_SortAndLanguage__lahQR .common_sort__z_v6q'))
                )
            except:
                print("드롭다운 메뉴를 찾을 수 없습니다. 리뷰가 없는 페이지일 수 있습니다.")
                return  # 드롭다운 메뉴가 없으면 함수 종료
        
           # 드롭다운 메뉴 클릭하여 열기
            sort_dropdown = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.common_SortAndLanguage__lahQR .common_sort__z_v6q'))
            )
            sort_dropdown.click()
            
            # 평점 낮은 순 선택
            lowest_rating_option = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.common_SortAndLanguage__lahQR .common_select__jY2oE option[value="worst"]'))
            )
            lowest_rating_option.click()
            
            # 평점 낮은 순 정렬 후 리뷰 리스트가 나타나는지 확인
            try:
                WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.common_ReviewList__XlcWm'))
                )
                print("정렬 기준을 평점 낮은 순으로 변경했습니다.")
            except:
                print("리뷰 리스트를 찾을 수 없습니다. 리뷰가 없는 페이지일 수 있습니다.")
                return  # 리뷰 리스트가 없으면 함수 종료

        else:
            print("이용후기 탭을 찾을 수 없습니다.")
            return  # 이용후기 탭이 없는 경우 함수 종료
        
    except Exception as e:
        print(f"정렬 변경 오류 발생: {e}")

In [147]:
# 데이터 수집 함수 호출
get_hotel_data()

  7%|▋         | 1/14 [00:02<00:38,  3.00s/it]

정렬 기준을 평점 낮은 순으로 변경했습니다.


 14%|█▍        | 2/14 [00:06<00:36,  3.01s/it]

정렬 기준을 평점 낮은 순으로 변경했습니다.


 21%|██▏       | 3/14 [00:11<00:45,  4.15s/it]

...

이용후기 탭을 찾을 수 없습니다.


 93%|█████████▎| 13/14 [00:33<00:03,  3.26s/it]

드롭다운 메뉴를 찾을 수 없습니다. 리뷰가 없는 페이지일 수 있습니다.


100%|██████████| 14/14 [00:35<00:00,  2.56s/it]

이용후기 탭을 찾을 수 없습니다.
페이지네이션의 다음 버튼이 비활성화되었습니다. 데이터 수집을 종료합니다.


In [148]:
df = pd.DataFrame(review_data)
df

,호텔이름,호텔성급,부킹닷컴평점,호텔스컴바인평점,트립어드바이저평점,리뷰,링크
0,부천 고려 호텔,3성급,7.2,8,3.5,"""최고""",https://hotels.naver.com/hotels/N1658848?adult...
1,폴라리스 호텔,3급,7.9,8.5,3.5,"""매우 좋음"" 슬리퍼가 없었던 점이 불편했어요.",https://hotels.naver.com/hotels/N1711337?adult...
2,부천 플로체 호텔,3급,0,7.8,4.5,"""매우 좋음"" 슬리퍼가 없었던 점이 불편했어요.",https://hotels.naver.com/hotels/N1714774?adult...
3,호텔 제니스,1성급,6.8,7.6,3.5,"""만족"" 조식 이좋았어요",https://hotels.naver.com/hotels/N2004268?adult...
4,B-Stay 부천호텔,2성급,0,8.5,0,"""만족"" 조식 이좋았어요",https://hotels.naver.com/hotels/N4479658?adult...
5,호텔 코보스,3급,0,7.9,4,"""Recommended""",https://hotels.naver.com/hotels/N4012129?adult...
6,멀티 부티크 호텔 K,2급,0,0,3,"""Recommended""",https://hotels.naver.com/hotels/N3005324?adult...
7,Sr호텔,2급,7.8,7.7,3.5,"""매우 좋음""",https://hotels.naver.com/hotels/N3232576?adult...
8,씨티 호텔,3급,0,6.4,0,"""매우 좋음""",https://hotels.naver.com/hotels/N4301844?adult...
9,에버 관광호텔,3급,0,6.8,4,"""프리미엄 룸""",https://hotels.naver.com/hotels/N1714772?adult...
